# Sudo code:

## Loop City Names: ["Vancouver", "Abbotsford", "Toronto"]

First loop is "Vancouver"

    * Post numbers are obtained all of the Vancouver regions post numbers. ['van', 'bnc', 'rds', 'nvn', 'rch', 'pml']

   

   ### Process to obtain the data:
    1. Get the number of posts.
        * define city name.
        * create url with get_url function
        * requests.get(url) to get response
        * use BeautifuSoup to get entire posts text
        * Find total number of posts then return 
    2. Use the total number of posts to get each pages first post number (119 posts per page)
        * List name pages e.g. [0, 120, 240, 360, ........]
    3. Scrape the information I need and store in the list
        * loop each pages list, it opens the page starting with the posts number
        * Create function to obtain informations.
            This will scrape each post, then store the information. 
    4. Creat DataFrame with the information I scraped.
        * Store inthe list of empty DataFrames 



In [1]:
class Url:
    def __init__(self, city_name):
        self.city_name = city_name
    
        
    def get_url(self, num_post, sub_city=None):
        if self.city_name == 'Abbotsford':
            url = f'https://{self.city_name.lower()}.craigslist.org/search/apa?s={num_post}sort=date&bundleDuplicates=1&min_price=99max_price=&availabilityMode=0&sale_date=all+dates'
        if sub_city is not None:
            sub_city = sub_city + '/'
            url = f'https://{self.city_name.lower()}.craigslist.org/search/{sub_city.lower()}apa?s={num_post}sort=date&bundleDuplicates=1&min_price=99max_price=&availabilityMode=0&sale_date=all+dates'
        return url   

In [35]:
from requests import get
from bs4 import BeautifulSoup
from time import sleep
import numpy as np
import pandas as pd

def get_search_number(url, city):
    response = get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    posts = soup.find_all('li', class_='result-row')
    results_num = soup.find('div', class_='search-legend')
    results_total = int(results_num.find('span', class_='totalcount').text)
    print(f"Total number of search result in {city}: {results_total}")
    return results_total

def scrape(city, total_num_posts):
    """
    city: ['Vancouver', 'Abbotsford', 'Toronto']
    total_num_posts: Total number of posts that got in get_search_number.
    """
    pages = np.arange(0, total_num_posts+1, 120)
        
    iterations = 0

    post_date = []
    post_cities = []
    num_bedroom = []
    sqfts = []
    prices = []
    post_titles = []
    post_links = []

    for page in pages:
     
        # get request
        url = Url(city)
        url_ = url.get_url(num_post=page)
        response = get(url_)
    
        sleep(1)
    
        # throw warning for status code that are not 200
        if response.status_code != 200:
            warn(f'Request: {requests}; Status code: {response.status_code}')
        
        soup = BeautifulSoup(response.text, 'html.parser')
    
        posts = soup.find_all('li', class_='result-row')
    
    
        # Extract data 
        for post in posts:
        
            if post.find('span', class_ = 'result-hood') is not None:
            
                # Posting date
                post_datetime = post.find('time', class_='result-date')['datetime']
                post_date.append(post_datetime)
            
                # Neighbourhoods
                post_city = post.find('span', class_='result-hood').text.strip('( )')
                post_cities.append(post_city)
            
                # title text
                post_title = post.find('a', class_='result-title hdrlnk')
                post_title_text = post_title.text
                post_titles.append(post_title_text)
            
                # Post price in integer
                price = int(post.find('span', class_='result-price').text.split('$')[1].replace(',', ''))
                prices.append(price)
            
                # Post link
                post_link = post_title['href']
                post_links.append(post_link)
            
                if post.find('span', class_ = 'housing') is not None:
                
                    #if the first element is accidentally square footage
                    if 'ft2' in post.find('span', class_ = 'housing').text.split()[0]:
                    
                        #make bedroom nan
                        bedroom_count = np.nan
                        num_bedroom.append(bedroom_count)
                    
                        #make sqft the first element
                        sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
                        sqfts.append(sqft)
                    
                    #if the length of the housing details element is more than 2
                    elif len(post.find('span', class_ = 'housing').text.split()) > 2:
                    
                        #therefore element 0 will be bedroom count
                        bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                        num_bedroom.append(bedroom_count)
                    
                        #and sqft will be number 3, so set these here and append
                        sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
                        sqfts.append(sqft)
                    
                    #if there is num bedrooms but no sqft
                    elif len(post.find('span', class_ = 'housing').text.split()) == 2:
                    
                        #therefore element 0 will be bedroom count
                        bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                        num_bedroom.append(bedroom_count)
                    
                        #and sqft will be number 3, so set these here and append
                        sqft = np.nan
                        sqfts.append(sqft)                    
                
                    else:
                        bedroom_count = np.nan
                        num_bedroom.append(bedroom_count)
                
                        sqft = np.nan
                        sqfts.append(sqft)
                
                #if none of those conditions catch, make bedroom nan, this won't be needed    
                else:
                    bedroom_count = np.nan
                    num_bedroom.append(bedroom_count)
                
                    sqft = np.nan
                    sqfts.append(sqft)
                #    num_bedroom.append(bedroom_count)
                
                #    sqft = np.nan
                #    sqfts.append(sqft)
            iterations += 1
    print(f'{iterations} data scraped.')
    
    # create DataFrame
    df = pd.DataFrame({'Post Datetime': post_date,
                       'Post Title': post_titles,
                       'Post URL': post_links,
                      'Neighborhood': post_cities,
                      'Bedroom': num_bedroom,
                      'SQFT': sqfts,
                      'Price': prices})
    return df
    
def num_of_posts(city_name):
    """
    Get number of posts in each reagions. 
    Return: list of page_nums.
    """
    page_nums = []
    
    if city_name == 'Abbotsford':
        url = Url(city_name)
        url_ = url.get_url(0)
        print(city_name)
        results_total = get_search_number(url_, city_name)
        
        df = scrape()
        
        page_nums.append(results_total)
    elif city_name == 'Vancouver':
        sub_city_names = ['van', 'bnc', 'rds', 'nvn', 'rch', 'pml']
        for sub_city in sub_city_names: 
            url = Url(city_name)
            url_ = url.get_url(0, sub_city)
            print(sub_city)
            results_total = get_search_number(url_, city_name)
            page_nums.append(results_total)
    elif city_name == 'Toronto':
        sub_city_names = ['tor', 'bra', 'drh', 'mss', 'oak', 'yrk']
        for sub_city in sub_city_names:
            url = Url(city_name)
            url_ = url.get_url(0, sub_city)
            print(sub_city)
            results_total = get_search_number(url_, city_name)
            page_nums.append(results_total)
    return page_nums

In [36]:
page_nums = num_of_posts('Toronto')

tor
Total number of search result in Toronto: 3000
bra
Total number of search result in Toronto: 56
drh
Total number of search result in Toronto: 47
mss
Total number of search result in Toronto: 242
oak
Total number of search result in Toronto: 9
yrk
Total number of search result in Toronto: 108


In [28]:
page_nums

[3000, 1313, 1821, 733, 370, 859, 1542, 3000, 56, 46, 242, 9, 108]

In [23]:
get_search_number('https://vancouver.craigslist.org/search/van/apa?s=0sort=date&bundleDuplicates=1&min_price=99max_price=&availabilityMode=0&sale_date=all+dates', 'Vancouver')

Total number of search result in Vancouver: 108


108